In [14]:
# !pip3 install langchain_community

In [15]:
from langchain_community.llms import Ollama

In [16]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man on the moon was Neil Armstrong. He stepped foot on the lunar surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" after becoming the first person to walk on the moon.'

In [17]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the categories for each of the expenses you provided:\n\nSpotify AB by Adyen - Entertainment\nBeta Boulders Ams Amsterdam Nld - Sports\nISS Catering Services De Meern - Food and Beverage\nVishandel Sier AMSTELVEEN - Food and Beverage\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Convenience Store\nBistro Bar Amsterdam - Food and Beverage'

### Read transaction data

In [18]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

/var/folders/4y/20c60c3x3476c1_zstk9vzj40000gn/T/ipykernel_32108/3553452196.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [19]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [20]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [21]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [22]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['Hello - World', 'Hello - there!'])

ResponseChecks(data=None)

In [23]:
def categorize_transactions(transaction_names, llm):
    original_response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = original_response.split('\n')

    # Keep only the lines with categories (removing the explaination lines at the beginning and end of the response)
    response = [item for item in response if item != '' and ' - ' in item]

    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [24]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Vishandel Sier AMSTELVEEN - Food', '5. Etos AMSTERDAM NLD - Food', '6. Bistro Bar Amsterdam - Food, Drink']


,Transaction vs category,Transaction,Category
0,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
1,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
2,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
3,4. Vishandel Sier AMSTELVEEN - Food,4. Vishandel Sier AMSTELVEEN,Food
4,5. Etos AMSTERDAM NLD - Food,5. Etos AMSTERDAM NLD,Food
5,"6. Bistro Bar Amsterdam - Food, Drink",6. Bistro Bar Amsterdam,"Food, Drink"


In [25]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

['1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Food', '5. Selling Paintings - Art/Craft', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Shopping', '8. Consulting - Professional Services', '9. Aidsfonds - Charity', '10. Tls Bv Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Groceries', '12. Beta Boulders Ams Amsterdam - Fitness', '13. Salary - Personal Expenses', '14. Bouldermuur Bv Amsterdam - Professional Services', '15. Birtat Restaurant Amsterdam - Dining', '16. Freelancing - Personal Expenses', '17. Tikkie - Financial Services', '18. Blogging - Personal Expenses', '19. Taxi Utrecht - Transportation', '20. Apple Services - Technology', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Entertainment']


In [26]:
categories_df_all.head()

,Transaction vs category,Transaction,Category
0,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
1,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
2,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
3,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food
4,5. Selling Paintings - Art/Craft,5. Selling Paintings,Art/Craft


In [27]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [28]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Taxes', 'Groceries', 'Housing', 'Food', 'Art/Craft',
       'Entertainment', 'Shopping', 'Professional Services', 'Charity',
       'Transportation', 'Fitness', 'Personal Expenses', 'Dining',
       'Financial Services', 'Technology'], dtype=object)

In [29]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [30]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
0,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
1,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
2,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
3,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food and Drinks
4,5. Selling Paintings - Art/Craft,5. Selling Paintings,Art/Craft
5,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
6,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping
7,8. Consulting - Professional Services,8. Consulting,Services
8,9. Aidsfonds - Charity,9. Aidsfonds,Charity
9,10. Tls Bv Inz Ov-Chipkaart - Transportation,10. Tls Bv Inz Ov-Chipkaart,Transportation


In [31]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [32]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)